In [ ]:
import LOTlib3
from LOTlib3.Grammar import Grammar
from LOTlib3.Eval import primitive
from LOTlib3.Hypotheses.LOTHypothesis import LOTHypothesis
from LOTlib3.Hypotheses.Likelihoods.BinaryLikelihood import BinaryLikelihood
from LOTlib3.Miscellaneous import q
from LOTlib3.DataAndObjects import FunctionData
from LOTlib3.Samplers.MetropolisHastings import MetropolisHastingsSampler
from LOTlib3.TopN import TopN

import numpy as np
from functools import cache
import builtins
from copy import copy
from itertools import product
from pprint import pprint

from copy import deepcopy

In [151]:
n_objects = 4
n_agents = 5
n_batches = 3
batches_size = 3
history_size = 4
expr_nest_prob = 0.25

In [173]:
run_simulation(
    n_agents, 
    n_objects, 
    n_batches, 
    batches_size, 
    history_size, 
    define_grammar(n_objects, n_agents, expr_nest_prob),
    log=True
)

game:
 [[3 4 0 2 1]
 [1 2 4 3 0]
 [3 1 2 0 4]
 [1 2 3 0 4]]
prefs:
 [[2 0 3 1]
 [2 3 0 1]
 [3 0 2 1]
 [2 0 1 3]
 [2 0 3 1]]
actions:  [1 1 1 3 1]
[-1.  1. -1.  1. -1.]

game:
 [[3 2 1 4 0]
 [4 2 0 1 3]
 [4 2 3 1 0]
 [2 4 3 1 0]]
prefs:
 [[2 0 3 1]
 [2 3 0 1]
 [3 0 2 1]
 [2 0 1 3]
 [2 0 3 1]]
actions:  [1 1 1 3 1]
[-1. -1. -1.  1.  1.]

game:
 [[1 3 2 0 4]
 [4 3 0 1 2]
 [2 1 4 3 0]
 [0 3 2 1 4]]
prefs:
 [[2 0 3 1]
 [2 3 0 1]
 [3 0 2 1]
 [2 0 1 3]
 [2 0 3 1]]
actions:  [1 1 1 3 1]
[-1. -1. -1.  1.  1.]

game:
 [[0 2 3 4 1]
 [1 4 2 0 3]
 [2 0 3 1 4]
 [2 1 0 3 4]]
prefs:
 [[2 0 3 1]
 [2 3 0 1]
 [3 0 2 1]
 [2 0 1 3]
 [2 0 3 1]]
actions:  [1 1 1 3 1]
[-1.  1. -1.  1. -1.]

Done with 0:0:3
0
topN not yet defined!

1
topN not yet defined!

2
topN not yet defined!

3
topN not yet defined!

4
topN not yet defined!

game:
 [[1 2 0 3 4]
 [3 4 0 1 2]
 [2 0 3 1 4]
 [4 0 2 1 3]]
prefs:
 [[1 2 0 3]
 [2 1 0 3]
 [2 0 1 3]
 [3 2 0 1]
 [1 3 2 0]]
actions:  [3 3 3 1 0]
[ 1. -1. -1.  1.  1.]

game:
 [[4 2 1

KeyboardInterrupt: 

In [178]:
grammar = define_grammar(4, 2, 0.25)

grammar.generate()

[(lambda game, own_index, mem, acts, preds: f_if(f_not(f_equals('a1', f_self(own_index))), lambda : f_predict('a0', 'o2', preds), lambda : f_predict('a1', 'o3', preds))), (lambda game, own_index, mem, acts: f_remove('a0', f_self(own_index), mem))]

# Library compression tests

In [3]:
n_objects = 5
n_agents = 5
expr_nest_prob = .25

In [105]:
"""
programs consist of a move prediction function and of a memory update function.
the memory structure is a set of strings for each agent.
the stored strings are built from the alphabet {z, names of agents, names of objects}
the prediction function calls "predict(agent,object)" and then any agents not predicted are given a random prediction
the update function calls "add(agent, string)" and "remove(agent, string)" 
to add/remove string from the set corresponding to agent
"""
grammar = Grammar(start='INF')

grammar.add_rule(
    'INF',
    # me is the agent's own ID
    'list(%s)(%s)',
    ['P_EXPR', 'M_EXPR'],
    1
)

#### define data strings
grammar.add_rule('STR', 'plus(%s)(%s)', ['STR','STR'], 0.5)
grammar.add_rule('STR','%s', ['OBJ'], 1.0)
grammar.add_rule('STR','%s', ['AG'], 1.0)
grammar.add_rule('STR','z', None, 1.0)

for i in range(n_objects):
    grammar.add_rule('OBJ', 'o' + str(i), None, 3/n_objects)

for i in range(n_agents):
    grammar.add_rule('AG', 'a' + str(i), None, 3/n_agents)

#### define boolean stuff, top-level variables
grammar.add_rule('BOOL', 'f_and(%s)(%s)', ['BOOL', 'BOOL'], 0.5)
grammar.add_rule('BOOL', 'f_not(%s)', ['BOOL'], 0.5)
grammar.add_rule('BOOL', 'f_equals(%s)(%s)', ['AG','AG'],0.5)
grammar.add_rule('BOOL', 'f_equals(%s)(%s)', ['OBJ','OBJ'],0.5)
grammar.add_rule('BOOL', 'f_equals(%s)(%s)', ['STR','STR'],0.5)

grammar.add_rule('GAME', 'game', None, 1)
grammar.add_rule('AG', 'ag', None, 1)
grammar.add_rule('MEM', 'mem', None, 1)
grammar.add_rule('ACTS','acts', None, 1)

#### ways for program to look at game, memory, acts
grammar.add_rule('BOOL', 'f_closerthan(%s)(%s)(%s)(%s)', ['AG','AG','OBJ','GAME'], 1.0)
grammar.add_rule('BOOL', 'f_chose(%s)(%s)(%s)', ['AG','OBJ','ACTS'], 1)
grammar.add_rule('BOOL', 'f_in(%s)(%s)(%s)', ['STR', 'MEM', 'AG'], 1)

#### define program logic for prediction program
grammar.add_rule('P_EXPR', 'f_if(%s)(%s)(%s)', ['BOOL','P_EXPR','P_EXPR'], expr_nest_prob)
grammar.add_rule('P_EXPR', 'f_predict(%s)(%s)', ['AG','OBJ'],1.0)

##  iterate over agents
grammar.add_rule('P_EXPR', 'forallagents', ['PA_FUNC'], expr_nest_prob)
grammar.add_rule('PA_FUNC', 'lambda', ['P_EXPR'],1.0, bv_type='AG',bv_prefix='x')

##  iterate over objects
grammar.add_rule('P_EXPR', 'forallobjects', ['PO_FUNC'], expr_nest_prob)
grammar.add_rule('PO_FUNC', 'lambda', ['P_EXPR'],1.0, bv_type='OBJ',bv_prefix='x')

## iterate over strings in memory[agent]
grammar.add_rule('P_EXPR','forallstrings(%s)(%s)',['AG','PS_FUNC'], expr_nest_prob)
grammar.add_rule('PS_FUNC', 'lambda', ['P_EXPR'], 1.0, bv_type='STR',bv_prefix='x')

#### define program logic for memory program
grammar.add_rule('M_EXPR', 'f_if(%s)(%s)(%s)',['BOOL', 'M_EXPR', 'M_EXPR'], expr_nest_prob)
grammar.add_rule('M_EXPR', 'f_write(%s)(%s)',['AG','STR'],1.0)
grammar.add_rule('M_EXPR', 'f_remove(%s)(%s)',['AG','STR'],1.0)

##  iterate over agents
grammar.add_rule('M_EXPR', 'forallagents', ['MA_FUNC'], expr_nest_prob)
grammar.add_rule('MA_FUNC', 'lambda', ['M_EXPR'],1.0, bv_type='AG',bv_prefix='x')

##  iterate over objects
grammar.add_rule('M_EXPR', 'forallobjects', ['MO_FUNC'],expr_nest_prob)
grammar.add_rule('MO_FUNC', 'lambda', ['M_EXPR'],1.0, bv_type='OBJ',bv_prefix='x')

## iterate over strings in memory[agent]
grammar.add_rule('M_EXPR','forallstrings(%s)(%s)',['AG','MS_FUNC'], expr_nest_prob)
grammar.add_rule('MS_FUNC', 'lambda', ['M_EXPR'], 1.0, bv_type='STR',bv_prefix='x')

MS_FUNC -> lambda['M_EXPR']	w/ p=1.0,	BV:STR;None;x

In [106]:
from LOTlib3.StitchBindings import stitch_to_python, python_to_stitch, bindings

In [81]:
from stitch_core import compress

In [109]:
xs = [
    grammar.generate()
    for _ in range(100)
]
print(xs[0])

list(f_predict(a4)(o2))(f_remove(ag)(a2))


In [110]:
xs_stitch = [
    python_to_stitch.python_to_stitch(x.__str__())
    for x in xs
]

In [119]:
results = compress(
    xs_stitch,
    iterations=10
)

In [120]:
results.abstractions

[fn_0(#0,#1) := (list (f_predict #1 #0)),
 fn_1(#0) := (forallagents (lam (f_predict $0 #0))),
 fn_2(#0) := (forallobjects (lam (f_predict #0 $0))),
 fn_3() := (f_remove ag),
 fn_4() := (f_remove a2),
 fn_5(#0,#1) := (f_if (f_in #1 mem #0)),
 fn_6(#0,#1) := (f_if (f_chose #1 #0 acts)),
 fn_7() := (fn_0 o2),
 fn_8() := (f_write ag),
 fn_9() := (fn_0 o4)]

In [113]:
python_rewritten = [
    bindings.abstraction_to_python(x.__str__())
    for x
    in results.abstractions
]

In [ ]:
grammar.add_rule(

In [114]:
python_rewritten

['lambda y0: lambda y1: list(f_predict(y1)(y0))',
 'lambda y0: forallagents(lambda x1: (f_predict(x1)(y0)))',
 'lambda y0: forallobjects(lambda x1: (f_predict(y0)(x1)))',
 'f_remove(ag)',
 'f_remove(a2)',
 'lambda y0: lambda y1: f_if(f_in(y1)(mem)(y0))',
 'lambda y0: lambda y1: f_if(f_chose(y1)(y0)(acts))',
 'fn_0(o2)',
 'f_write(ag)',
 'fn_0(o4)']

In [122]:
bindings.compress_and_transform(
    xs_stitch,
    grammar,
    # how many hypotheses to compress per task
    n_per_task=5,
    eta_long=True,
    prefix='',
    best_names=None,
    # weight of new abstractions
    weight=1,
)

KeyError: 'x1'